In [40]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from pymongo import MongoClient
import requests
%load_ext autoreload
%autoreload 2
import pandas as pd
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from pymongo import MongoClient
import requests
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


mc = MongoClient()
capstone_db = mc['bit_scraping']
#artist_names  = capstone_db['artist_names']
artist_info = capstone_db['artist']

In [9]:
#gd.download_band_names(['OR'],'bands_in_ca.csv')

In [10]:
#gd.download_band_names(['OR','CA'],'bands_in_ca.csv')

In [6]:
artist_info.delete_many({})

In [3]:
data = pd.read_csv('events.csv')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103427 entries, 0 to 103426
Data columns (total 30 columns):
Unnamed: 0              103427 non-null int64
_id                     103427 non-null object
artist_facebook         64697 non-null object
artist_image            103427 non-null object
artist_name             103427 non-null object
artist_thumb            103427 non-null object
artist_url              103427 non-null object
artists_id              103427 non-null int64
bio                     77455 non-null object
genre                   85050 non-null object
home_town               64764 non-null object
mbid                    41035 non-null object
tracker_count           103427 non-null int64
upcoming_event_count    103427 non-null int64
offers                  103427 non-null object
datetime                103427 non-null object
on_sale_datetime        34191 non-null object
description             32091 non-null object
lineup                  103427 non-null object
event_i

In [28]:
df = data[['artists_id', 'venue_id']]

In [38]:
df.head(3)

,artists_id,venue_id,event_counts
0,14849750,12242,1
1,14849750,15607,1
2,14849750,23800,1


In [31]:
df['event_counts'] = 1

/Users/shiahlints/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
df.head(5)

,artists_id,venue_id,event_counts
0,14849750,12242,1
1,14849750,15607,1
2,14849750,23800,1
3,14849750,4271,1
4,14849750,16032,1


In [33]:
ratings_matrix = df.groupby(['artists_id', 'venue_id'], as_index = False).count()

In [34]:
dt = pd.DataFrame(ratings_matrix)

In [36]:
dt.head(5)

,artists_id,venue_id,event_counts
0,4,2145,1
1,4,18590,1
2,4,28176,1
3,7,1944,1
4,7,3203,1


In [41]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [42]:
sp_df = spark.createDataFrame(dt)

In [43]:
sp_df.show()

+----------+--------+------------+
|artists_id|venue_id|event_counts|
+----------+--------+------------+
|         4|    2145|           1|
|         4|   18590|           1|
|         4|   28176|           1|
|         7|    1944|           1|
|         7|    3203|           1|
|         7|    5160|           1|
|         7|    7045|           1|
|         7|   12463|           1|
|         7|   13845|           1|
|         7|   14862|           1|
|         7|   15279|           1|
|         7|   17487|           1|
|         7|   17913|           1|
|         7|   19230|           1|
|         7|   20502|           1|
|         7|   22184|           1|
|         7|   22796|           1|
|         7|   23130|           1|
|         7|   23274|           1|
|         7|   25258|           1|
+----------+--------+------------+
only showing top 20 rows



In [44]:
train, test = sp_df.randomSplit([0.8, 0.2], seed=42)

In [47]:
als_model = ALS(userCol='venue_id',
                itemCol='artists_id',
                ratingCol='event_counts',
                nonnegative=True,
                regParam=0.1,
                rank=5,
                implicitPrefs=True,
                alpha = 40)

In [48]:
recomender = als_model.fit(train)

In [59]:
rjj = 11270709

In [62]:
venues_in_seattle = np.unique(data[data['city'] == 'Seattle']['venue_id'])

In [75]:
rate_this = pd.DataFrame()

In [101]:
venues_in_seattle

array([  138,   200,   205,   265,   355,   367,   392,   393,   394,
         397,   448,   919,  1105,  1130,  1198,  1526,  1647,  1676,
        1808,  1858,  1970,  2108,  2247,  2253,  2404,  2429,  2435,
        2437,  2498,  2987,  2989,  2998,  3086,  3152,  3154,  3155,
        3156,  3178,  3215,  3272,  3273,  3274,  3275,  3282,  3393,
        3404,  3501,  3630,  3723,  3767,  3769,  3770,  3771,  3772,
        4170,  4171,  4193,  4239,  4405,  4600,  4661,  4662,  4691,
        4726,  4921,  5008,  5166,  5239,  5459,  5460,  5461,  5462,
        5463,  5464,  5465,  5466,  5467,  5607,  5611,  5688,  5728,
        5749,  6241,  6668,  6744,  6809,  6894,  7100,  7144,  7433,
        7497,  7498,  7549,  7605,  7668,  7679,  7680,  7682,  7683,
        7864,  7866,  7867,  7868,  7869,  7919,  8027,  8105,  8106,
        8107,  8559,  8560,  8561,  8562,  8563,  8688,  8706,  8751,
        8752,  8837,  8838,  8916,  8963,  9143,  9168,  9197,  9203,
        9348,  9349,

In [76]:
rate_this['venue_id'] = venues_in_seattle

In [77]:
rate_this['artists_id'] = rjj

In [78]:
rate_this.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
venue_id      667 non-null int64
artists_id    667 non-null int64
dtypes: int64(2)
memory usage: 10.5 KB


In [80]:
rate_this = rate_this[['artists_id', 'venue_id']]

In [82]:
rate_this_spark = spark.createDataFrame(rate_this)

In [83]:
rate_this_spark.show()

+----------+--------+
|artists_id|venue_id|
+----------+--------+
|  11270709|     138|
|  11270709|     200|
|  11270709|     205|
|  11270709|     265|
|  11270709|     355|
|  11270709|     367|
|  11270709|     392|
|  11270709|     393|
|  11270709|     394|
|  11270709|     397|
|  11270709|     448|
|  11270709|     919|
|  11270709|    1105|
|  11270709|    1130|
|  11270709|    1198|
|  11270709|    1526|
|  11270709|    1647|
|  11270709|    1676|
|  11270709|    1808|
|  11270709|    1858|
+----------+--------+
only showing top 20 rows



In [85]:
predictions = recomender.transform(rate_this_spark)

In [86]:
predictions.show()

+----------+--------+-------------+
|artists_id|venue_id|   prediction|
+----------+--------+-------------+
|  11270709|   18800| 5.8695718E-5|
|  11270709|   29601|  6.544181E-4|
|  11270709|   37263|  1.321246E-4|
|  11270709|   38422|          0.0|
|  11270709|   40107|          0.0|
|  11270709|   40386|          0.0|
|  11270709|     392|          0.0|
|  11270709|    8105|          0.0|
|  11270709|   19758|          0.0|
|  11270709|   19868|          0.0|
|  11270709|   32680|          NaN|
|  11270709|    4239|          0.0|
|  11270709|   13465|          0.0|
|  11270709|   23367|          0.0|
|  11270709|   14944|          0.0|
|  11270709|   31646|3.9469136E-11|
|  11270709|    2247|          0.0|
|  11270709|   14365|          NaN|
|  11270709|   21126|          0.0|
|  11270709|   17903|          0.0|
+----------+--------+-------------+
only showing top 20 rows



In [87]:
venues = pd.read_csv('venues.csv')

In [108]:
thing = predictions.dropna().sort('prediction',ascending=False)

In [109]:
predictionspd = thing.toPandas()

In [115]:
top_pridictions = predictionspd['venue_id'].values[0:10]

In [100]:
venues[venues['venue_id']== 4239]

,Unnamed: 0,venueidentifer,venue_name,country,region,city,genre,bio,latitude,longitude,venue_id
4239,4239,Black Lodge Seattle WA United States,Black Lodge,United States,WA,Seattle,"['Space Rock, Alternative, Post Hardcore']",['When great bands break up there is rarely an...,47.606209,-122.332071,4239


In [120]:
data[data['venue_id'].isin(top_pridictions)]

,Unnamed: 0,_id,artist_facebook,artist_image,artist_name,artist_thumb,artist_url,artists_id,bio,genre,...,event_url,band_name,venue_name,country,region,city,latitude,longitude,venueidentifer,venue_id
1023,1023,5b7718d5482deb63557f8d29,https://www.facebook.com/LillyHiatt,https://s3.amazonaws.com/bit-photos/large/7928...,Lilly Hiatt,https://s3.amazonaws.com/bit-photos/thumb/7928...,https://www.bandsintown.com/a/11801329?came_fr...,11801329,The new album 'Trinity Lane' is available Augu...,"Songs, Folk",...,https://www.bandsintown.com/e/1009523252?app_i...,Lilly Hiatt,Triple Door,United States,WA,Seattle,47.608433,-122.337944,Triple Door Seattle WA United States,38894
1048,1048,5b7718e6482deb63557f8d34,https://www.facebook.com/AMANDASHIRESMUSIC,https://s3.amazonaws.com/bit-photos/large/8692...,Amanda Shires,https://s3.amazonaws.com/bit-photos/thumb/8692...,https://www.bandsintown.com/a/184270?came_from...,184270,5'3 and hoping.,Americana,...,https://www.bandsintown.com/e/1009443259?app_i...,Amanda Shires,Triple Door,United States,WA,Seattle,47.608433,-122.337944,Triple Door Seattle WA United States,38894
2110,2110,5b771a03482deb63557f8e45,https://www.facebook.com/nydrock,https://s3.amazonaws.com/bit-photos/large/8547...,New Years Day,https://s3.amazonaws.com/bit-photos/thumb/8547...,https://www.bandsintown.com/a/1914?came_from=2...,1914,New Years Day is an American rock band formed ...,"Rock, Hard Rock",...,https://www.bandsintown.com/e/1009205389?app_i...,New Years Day,CenturyLink Field,United States,WA,Seattle,47.595146,-122.331601,CenturyLink Field Seattle WA United States,6809
2136,2136,5b771a06482deb63557f8e49,https://www.facebook.com/halestormrocks,https://s3.amazonaws.com/bit-photos/large/8726...,Halestorm,https://s3.amazonaws.com/bit-photos/thumb/8726...,https://www.bandsintown.com/a/4152?came_from=2...,4152,After scoring two top 10 singles (“I Get Off” ...,Rock,...,https://www.bandsintown.com/e/1009205393?app_i...,Halestorm,CenturyLink Field,United States,WA,Seattle,47.595146,-122.331601,CenturyLink Field Seattle WA United States,6809
2194,2194,5b771a0f482deb63557f8e50,https://www.facebook.com/officialinthismoment,https://s3.amazonaws.com/bit-photos/large/8259...,In This Moment,https://s3.amazonaws.com/bit-photos/thumb/8259...,https://www.bandsintown.com/a/1105?came_from=2...,1105,"Led by front woman Maria Brink, the Los Angele...","Metal, Rock, Metalcore, Alternative",...,https://www.bandsintown.com/e/1009205387?app_i...,In This Moment,CenturyLink Field,United States,WA,Seattle,47.595146,-122.331601,CenturyLink Field Seattle WA United States,6809
6075,6075,5b77224d482deb63557f9565,https://www.facebook.com/mirahmusic,https://s3.amazonaws.com/bit-photos/large/8786...,Mirah,https://s3.amazonaws.com/bit-photos/thumb/8786...,https://www.bandsintown.com/a/2517?came_from=2...,2517,Mirah Yom Tov Zeitlyn has been creating incorr...,"Alternative, Indie",...,https://www.bandsintown.com/e/100153198?app_id...,Mirah,Ballard Homestead,United States,WA,Seattle,47.606209,-122.332071,Ballard Homestead Seattle WA United States,3156
6443,6443,5b772377482deb63557f962d,https://www.facebook.com/10StringSymphony,https://s3.amazonaws.com/bit-photos/large/6292...,10 String Symphony,https://s3.amazonaws.com/bit-photos/thumb/6292...,https://www.bandsintown.com/a/3751993?came_fro...,3751993,Never underestimate the power of caffeine and ...,Progressive Acoustic Folk,...,https://www.bandsintown.com/e/1011106549?app_i...,10 String Symphony,Ballard Homestead,United States,WA,Seattle,47.677033,-122.386760,Ballard Homestead Seattle WA United States,3156
7001,7001,5b773381482deb63557f9bb7,NaN,https://s3.amazonaws.com/bit-photos/large/7867...,Colm Keegan,https://s3.amazonaws.com/bit-photos/thumb/7867...,https://www.bandsintown.com/a/5274630?came_fro...,5274630,Colm Keegan Singer/Entertainer/Online Educator...,NaN,...,https://www.bandsintown.com/e/100013161?app_id...,Colm Keegan,Ballard Homestead,United States,WA,Seattle,47.606209,-122.332071,Ballard Homest